In [9]:
import pandas as pd
import numpy as np
from collections import Counter

from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from utils import anderson, count_parameters, epoch, epoch_eval
from models import ResBlock, ODEfunc, ODENet, DEQfunc, DEQNet

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0);

In [5]:
def get_model(net, device, n_channels=32, n_inner_channels=32, kernel_size=3, 
              num_groups=8, adam=False, **kwargs):
    """
    Initialize ResNet, ODENet, or DEQNet with optimizer.
    """
    downsampling_layers = [
        nn.Conv1d(1, n_channels, kernel_size=3, bias=True, padding="same"),
        nn.BatchNorm1d(n_channels)
    ]

    if net == 'ResNet':
        feature_layers = [ResBlock(n_channels, n_inner_channels, kernel_size, num_groups) for _ in range(1)]
    elif net == 'ODENet':
        feature_layers = [ODENet(ODEfunc(n_channels, n_inner_channels, kernel_size, num_groups), **kwargs)]
    elif net == 'DEQNet':
        feature_layers = [DEQNet(DEQfunc(n_channels, n_inner_channels, kernel_size, num_groups), anderson, **kwargs)]
    else:
        return 0
        
    fc_layers = [
        nn.BatchNorm1d(n_channels), 
        nn.ReLU(inplace=True), 
        nn.AdaptiveAvgPool1d(1), 
        nn.Flatten(), 
        nn.Linear(n_channels, 5)
    ]

    model = nn.Sequential(*downsampling_layers, *feature_layers, *fc_layers)

    opt = optim.Adam(model.parameters(), lr=1e-3) if adam else optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

    return model.to(device), opt

In [10]:
np.random.seed(42)
path = 'data/'

# Load MIT-BIH data
mit_train = pd.read_csv(path + "mitdb_360_train.csv", header=None)
mit_test = pd.read_csv(path + "mitdb_360_test.csv", header=None)

y_train = mit_train[360]
X_train = mit_train.loc[:, :359]
y_test = mit_test[360]
X_test = mit_test.loc[:, :359]
print('Train before:', Counter(y_train))
print('Test before:', Counter(y_test), end='\n\n')

# Oversample training set
ros = RandomOverSampler(random_state=0)
X_train_oversampled, y_train_oversampled = ros.fit_resample(X_train, y_train)
print('Train oversampled:', Counter(y_train_oversampled), end='\n\n')

# Split a validation set
X_train, X_val, y_train, y_val = train_test_split(
                                    X_train_oversampled, 
                                    y_train_oversampled, 
                                    test_size=0.1, 
                                    random_state=42)
print('Train after:', Counter(y_train))
print('Val after:', Counter(y_val))
print('Test after:', Counter(y_test))

# Convert to 3D tensor
X_train, y_train, X_val, y_val, X_test, y_test = map(
    torch.from_numpy, 
    (X_train.values, y_train.values, 
     X_val.values, y_val.values, 
     X_test.values, y_test.values)
)
X_train = X_train.unsqueeze(1).float()
X_val = X_val.unsqueeze(1).float()
X_test = X_test.unsqueeze(1).float()
y_train = y_train.long()
y_val = y_val.long()
y_test = y_test.long()

# Batch size
bs = 128

# Dataloaders
train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=8)
val_ds = TensorDataset(X_val, y_val)
val_dl = DataLoader(val_ds, batch_size=bs * 2, shuffle=False, num_workers=8)
test_ds = TensorDataset(X_test, y_test)
test_dl = DataLoader(test_ds, batch_size=bs * 2, shuffle=False, num_workers=8)

Train before: Counter({0: 88069, 2: 7042, 4: 3625, 1: 3016, 3: 760})
Test before: Counter({0: 800, 1: 800, 2: 800, 4: 800, 3: 300})

Train oversampled: Counter({0: 88069, 1: 88069, 2: 88069, 4: 88069, 3: 88069})

Train after: Counter({0: 79358, 3: 79269, 4: 79264, 1: 79254, 2: 79165})
Val after: Counter({2: 8904, 1: 8815, 4: 8805, 3: 8800, 0: 8711})
Test after: Counter({0: 800, 1: 800, 2: 800, 4: 800, 3: 300})


In [ ]:
# Initialize ResNet
resnet, resnetopt = get_model(net='ResNet', device=device,
                              n_channels=32, n_inner_channels=32, 
                              kernel_size=3, num_groups=8, adam=True)

# Training options
max_epochs = 15
scheduler = optim.lr_scheduler.ReduceLROnPlateau(resnetopt, mode='min', factor=0.1, patience=5)

# Training loop
for i in range(max_epochs):
    epoch(train_dl, resnet, device, resnetopt, scheduler, epoch=i+1)
    epoch_eval(val_dl, resnet, device)

In [22]:
# Test set
print("Number of Parameters:", count_parameters(resnet), end='\n\n')
epoch_eval(test_dl, resnet, device)

Number of Parameters: 6693

Testing
    Test acc: 0.96
    Test loss: 0.122
              precision    recall  f1-score   support

           0       0.94      0.93      0.94       800
           1       0.96      0.95      0.95       800
           2       0.98      0.95      0.96       800
           3       0.86      1.00      0.92       300
           4       1.00      1.00      1.00       800

    accuracy                           0.96      3500
   macro avg       0.95      0.96      0.96      3500
weighted avg       0.96      0.96      0.96      3500

[[746  28   9  17   0]
 [ 39 758   2   1   0]
 [  9   3 758  30   0]
 [  0   0   1 299   0]
 [  0   0   1   0 799]]



In [ ]:
# Initialize ODENet
odenet, odeopt = get_model(net='ODENet',device=device,
                           n_channels=32, n_inner_channels=32, 
                           kernel_size=3, num_groups=8, adam=True,
                           rtol=1e-3, atol=1e-3)

# Training Options
max_epochs = 15
scheduler = optim.lr_scheduler.ReduceLROnPlateau(odeopt, mode='min', factor=0.1, patience=5)

# Training loop
for i in range(max_epochs):
    epoch(train_dl, odenet, device, odeopt, scheduler, epoch=i+1)
    epoch_eval(val_dl, odenet, device)

In [21]:
# Test set
print("Number of Parameters:", count_parameters(odenet), end='\n\n')
epoch_eval(test_dl, odenet, device)

Number of Parameters: 6885

Testing
    Test acc: 0.921
    Test loss: 0.214
              precision    recall  f1-score   support

           0       0.87      0.87      0.87       800
           1       0.94      0.87      0.90       800
           2       0.95      0.94      0.94       800
           3       0.77      0.94      0.85       300
           4       1.00      1.00      1.00       800

    accuracy                           0.92      3500
   macro avg       0.90      0.92      0.91      3500
weighted avg       0.92      0.92      0.92      3500

[[696  43  18  43   0]
 [ 84 696   8  12   0]
 [ 14   3 752  29   2]
 [  3   1  15 281   0]
 [  0   0   1   0 799]]



In [ ]:
# Inilialize DEQNet
deqnet, deqopt = get_model(net='DEQNet', device=device,
                           n_channels=32, n_inner_channels=32, 
                           kernel_size=3, num_groups=8, adam=True,
                           tol=1e-3, max_iter=50, m=5)

# Trainin options
max_epochs = 15
scheduler = optim.lr_scheduler.ReduceLROnPlateau(deqopt, mode='min', factor=0.1, patience=5)

# Training loop
for i in range(max_epochs):
    epoch(train_dl, deqnet, device, deqopt, scheduler, epoch=i+1)
    epoch_eval(val_dl, deqnet, device)

In [25]:
# Test set
print("Number of Parameters:", count_parameters(deqnet), end='\n\n')
epoch_eval(test_dl, deqnet, device)

Number of Parameters: 6757

Testing
    Test acc: 0.927
    Test loss: 0.21
              precision    recall  f1-score   support

           0       0.85      0.91      0.88       800
           1       0.95      0.85      0.90       800
           2       0.96      0.94      0.95       800
           3       0.84      0.96      0.89       300
           4       0.99      1.00      1.00       800

    accuracy                           0.93      3500
   macro avg       0.92      0.93      0.92      3500
weighted avg       0.93      0.93      0.93      3500

[[726  35  11  23   5]
 [101 684  10   5   0]
 [ 24   1 748  27   0]
 [  4   0   9 287   0]
 [  0   0   1   0 799]]

